# CH-06 個股分析機器人

### 1️⃣ 安裝及匯入套件

In [1]:
!pip install openai
!pip install yfinance==0.2.38
from openai import OpenAI, OpenAIError # 串接 OpenAI API
import yfinance as yf
import pandas as pd # 資料處理套件
import numpy as np
import datetime as dt # 時間套件
import requests
from bs4 import BeautifulSoup

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.52
    Uninstalling yfinance-0.2.52:
      Successfully uninstalled yfinance-0.2.52


### 2️⃣ 輸入 Gemini API KEY

In [2]:
from google.colab import userdata

client = OpenAI(
    api_key=userdata.get('GEMINI_API_KEY'),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

### 3️⃣ 取得股價資料

In [3]:
# 從 yfinance 取得一周股價資料
def stock_price(stock_id="大盤", days = 10):
  if stock_id == "大盤":
    stock_id="^TWII"
  else:
    stock_id += ".TW"

  end = dt.date.today() # 資料結束時間
  start = end - dt.timedelta(days=days) # 資料開始時間
  # 下載資料
  df = yf.download(stock_id, start=start)

  # 更換列名
  df.columns = ['開盤價', '最高價', '最低價',
                '收盤價', '調整後收盤價', '成交量']

  data = {
    '日期': df.index.strftime('%Y-%m-%d').tolist(),
    '收盤價': df['收盤價'].tolist(),
    '每日報酬': df['收盤價'].pct_change().tolist(),
    '漲跌價差': df['調整後收盤價'].diff().tolist()
    }

  return data

print(stock_price("2330"))

[*********************100%%**********************]  1 of 1 completed

{'日期': ['2025-02-05', '2025-02-06', '2025-02-07', '2025-02-10', '2025-02-11', '2025-02-12', '2025-02-13', '2025-02-14'], '收盤價': [1110.0, 1115.0, 1125.0, 1105.0, 1110.0, 1100.0, 1090.0, 1060.0], '每日報酬': [nan, 0.0045045045045044585, 0.008968609865470878, -0.01777777777777778, 0.004524886877828038, -0.009009009009009028, -0.009090909090909038, -0.02752293577981646], '漲跌價差': [nan, 5.0, 10.0, -20.0, 5.0, -10.0, -10.0, -30.0]}


### 4️⃣ 取得基本面資料

In [4]:
# 基本面資料
def stock_fundamental(stock_id= "大盤"):
  if stock_id == "大盤":
      return None

  stock_id += ".TW"
  stock = yf.Ticker(stock_id)

  # 營收成長率
  quarterly_revenue_growth = np.round(stock.quarterly_financials.loc["Total Revenue"].pct_change(-1).dropna().tolist(), 2)

  # 每季EPS
  quarterly_eps = np.round(stock.quarterly_financials.loc["Basic EPS"].dropna().tolist(), 2)

  # EPS季增率
  quarterly_eps_growth = np.round(stock.quarterly_financials.loc["Basic EPS"].pct_change(-1).dropna().tolist(), 2)

  # 轉換日期
  dates = [date.strftime('%Y-%m-%d') for date in stock.quarterly_financials.columns]

  data = {
      '季日期': dates[:len(quarterly_revenue_growth)],
      '營收成長率': quarterly_revenue_growth.tolist(),
      'EPS': quarterly_eps[0:3].tolist(),
      'EPS 季增率': quarterly_eps_growth[0:3].tolist()
  }

  return data

print(stock_fundamental("2330"))

{'季日期': ['2024-12-31', '2024-09-30', '2024-06-30', '2024-03-31', '2023-12-31', '2023-09-30'], '營收成長率': [0.14, 0.13, 0.14, -0.05, 0.0, 0.0], 'EPS': [14.45, 12.55, 9.56], 'EPS 季增率': [0.15, 0.31, 0.1]}


<ipython-input-4-3ab454e80453>:10: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  quarterly_revenue_growth = np.round(stock.quarterly_financials.loc["Total Revenue"].pct_change(-1).dropna().tolist(), 2)
<ipython-input-4-3ab454e80453>:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  quarterly_revenue_growth = np.round(stock.quarterly_financials.loc["Total Revenue"].pct_change(-1).dropna().tolist(), 2)
<ipython-input-4-3ab454e80453>:16: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any no

### 5️⃣ 取得新聞資料

In [5]:
# 新聞資料
def stock_news(stock_name ="大盤"):
  if stock_name == "大盤":
    stock_name="台股 -盤中速報"

  data=[]
  # 取得 Json 格式資料
  json_data = requests.get(f'https://ess.api.cnyes.com/ess/api/v1/news/keyword?q={stock_name}&limit=5&page=1').json()

  # 依照格式擷取資料
  items=json_data['data']['items']
  for item in items:
      # 網址、標題和日期
      news_id = item["newsId"]
      title = item["title"]
      publish_at = item["publishAt"]
      # 使用 UTC 時間格式
      utc_time = dt.datetime.utcfromtimestamp(publish_at)
      formatted_date = utc_time.strftime('%Y-%m-%d')
      # 前往網址擷取內容
      url = requests.get(f'https://news.cnyes.com/'
                        f'news/id/{news_id}').content
      soup = BeautifulSoup(url, 'html.parser')
      p_elements=soup .find_all('p')
      # 提取段落内容
      p=''
      for paragraph in p_elements[4:]:
          p+=paragraph.get_text()
      data.append([stock_name, formatted_date ,title,p])
  return data

print(stock_news("台積電"))

[['台積電', '2025-02-15', '〈熱門股〉被忽視的機器人概念股 和大彈近17%寫7個月新高價', '和大董事長沈國榮日前指出，傳產業者近二年真的是非常辛苦，但已度過最困難的時代，今年起將邁向更大發展，除了將切入半導體封裝版圖，也將拓展機器人市場。沈國榮強調，依目前的設備「完全有能力」切入人形機器人版圖，目前產能規劃可因應 80 億元營收的各業務總出貨所需，而嘉義新廠正如火如荼建置中。\xa0#帶量突破均線糾結100%#均線指標上攻100%#動能指標上漲股100%#偏弱機會股58.97%#指標剛跌破58.97%上一篇下一篇〈中美環球展望〉併購失敗的轉機 環球晶選擇一條難走但對的道路〈航運指數〉SCFI四大長程航線運價續跌 跌幅7-12%左右晶片關稅將揭曉，川普步步緊逼台積電何時止跌?外資賣超245億元 砍台積電逾2萬張 轉向敲進債券ETF俄烏停火：鋼鐵、散裝VS.晶片關稅：台股、台積電？買誰好？怎麼買？專家：注意這個指標\u200c\u200c\u200c'], ['台積電', '2025-02-15', '白宮官員：川普可能不支持英特爾美國廠由外國企業營運', '據《彭博》稍早援引知情人士報導，川普團隊最近曾與台積電高層會面，並提出兩家公司可能達成交易的想法，而台積電方面對此表示開放態度。白宮官員未直接回應川普團隊是否確實與台積電高層會面，並提出這項交易的構想。同時，該官員也未回應後續問題，即台積電是否可能透過在美國的合資企業參與英特爾的晶圓廠營運。目前這筆交易仍充滿不確定性，但若成真，可能會成為英特爾的財務生機。英特爾近年來在晶片製造領域的競爭力下滑，未能把握 AI 浪潮帶來的機會，同時又砸下數十億美元轉型為晶圓代工廠，這項轉型至今仍未見成效。英特爾股價周五收跌 2.2%，而台積電在美國掛牌的 ADR 則上漲約 1%。上一篇下一篇1〈美股盤後〉川普宣布對等關稅計畫 輝達漲超3% 四大指數全揚川普宣布對等關稅計畫 最快4月實施川普擬修改《晶片法案》條件 或延遲部分補助款〈台股盤前要聞〉外資趁漲賣脫台積電、日月光先進封測業績大進補、住展風向球轉黃藍燈〈能源盤後〉俄烏和談有望 緩解供應擔憂 原油連跌2日\u200c\u200c\u200c'], ['台積電', '2025-02-15', '<mark>台積電</mark>董事會、中美環球展望、台鋼集團擬

### 6️⃣ 爬取股號、股名對照表

In [6]:
# 取得全部股票的股號、股名
def stock_name():
  print("線上讀取股號、股名、及產業別")

  response = requests.get('https://isin.twse.com.tw/isin/C_public.jsp?strMode=2')
  url_data = BeautifulSoup(response.text, 'html.parser')
  stock_company = url_data.find_all('tr')

  # 資料處理
  data = [
      (row.find_all('td')[0].text.split('\u3000')[0].strip(),
        row.find_all('td')[0].text.split('\u3000')[1],
        row.find_all('td')[4].text.strip())
      for row in stock_company[2:] if len(row.find_all('td')[0].text.split('\u3000')[0].strip()) == 4
  ]

  df = pd.DataFrame(data, columns=['股號', '股名', '產業別'])

  return df

name_df = stock_name()

線上讀取股號、股名、及產業別


### 7️⃣ 取得股票名稱

In [7]:
# 取得股票名稱
def get_stock_name(stock_id, name_df):
    return name_df.set_index('股號').loc[stock_id, '股名']

print(name_df.head())
print("--------------------------")
print(get_stock_name("1417",name_df))

     股號  股名   產業別
0  1101  台泥  水泥工業
1  1102  亞泥  水泥工業
2  1103  嘉泥  水泥工業
3  1104  環泥  水泥工業
4  1108  幸福  水泥工業
--------------------------
嘉裕


### 8️⃣ 建構 Gemini 2.0 Flash 模型

In [8]:
# 建立 Gemini 2.0 Flash 模型
def get_reply(messages):
    try:
        response = client.chat.completions.create(
            model = "gemini-2.0-flash",
            messages = messages
        )
        reply = response.choices[0].message.content
    except OpenAIError as err:
        reply = f"發生 {err.type} 錯誤\n{err.message}"
    return reply

# 建立訊息指令(Prompt)
def generate_content_msg(stock_id, name_df):

    stock_name = get_stock_name(
        stock_id, name_df) if stock_id != "大盤" else stock_id

    price_data = stock_price(stock_id)
    news_data = stock_news(stock_name)

    content_msg = f'請依據以下資料來進行分析並給出一份完整的分析報告:\n'

    content_msg += f'近期價格資訊:\n {price_data}\n'

    if stock_id != "大盤":
        stock_value_data = stock_fundamental(stock_id)
        content_msg += f'每季營收資訊：\n {stock_value_data}\n'

    content_msg += f'近期新聞資訊: \n {news_data}\n'
    content_msg += f'請給我{stock_name}近期的趨勢報告,請以詳細、\
      嚴謹及專業的角度撰寫此報告,並提及重要的數字, reply in 繁體中文'

    return content_msg

# StockGPT
def stock_gpt(stock_id, name_df=name_df):
    content_msg = generate_content_msg(stock_id, name_df)

    msg = [{
        "role": "system",
        "content": f"你現在是一位專業的證券分析師, 你會統整近期的股價\
      、基本面、新聞資訊等方面並進行分析, 然後生成一份專業的趨勢分析報告"
    }, {
        "role": "user",
        "content": content_msg
    }]

    reply_data = get_reply(msg)

    return reply_data


### 9️⃣ 大盤趨勢報告

In [9]:
reply = stock_gpt(stock_id="大盤")
print(reply)

[*********************100%%**********************]  1 of 1 completed


## 台股近期趨勢分析報告 (2025/02/05 - 2025/02/15)

**報告日期：** 2025年2月16日

**1. 摘要：**

本報告旨在分析台股自2025年2月5日至2025年2月14日的走勢，並結合近期新聞資訊，評估市場趨勢與潛在風險。近期台股呈現震盪格局，雖有AI與機器人產業題材支撐，但受到國際情勢(如俄烏戰爭、關稅政策)與總體經濟數據(如美國零售銷售數據)等因素影響，市場情緒較為謹慎。2月14日，台股下跌246.8點，收在23152.61點，顯示市場存在一定壓力。

**2. 股價走勢分析：**

*   **整體表現：** 近期台股呈現震盪走勢，期間最高點為2月7日的23478.27點，最低點為2月5日的23161.58點。
*   **每日報酬率：** 從每日報酬率來看，正負交錯，顯示市場缺乏明確方向。其中，2月10日下跌0.96%，2月14日下跌1.05%，是近期較大的跌幅。
*   **漲跌價差：** 漲跌價差的波動也反映了市場的多空拉鋸。2月14日跌幅較大，顯示當日賣壓較重。

**關鍵數據：**

*   **期間漲幅：** 自2月5日至2月14日，台股下跌0.04%，整體表現疲弱。
*   **最大單日漲幅：** 2月7日，上漲0.69%。
*   **最大單日跌幅：** 2月14日，下跌1.05%。

**3. 基本面分析：**

*   **營收狀況：** 從新聞資訊來看，2025年1月份營收報告已陸續公布。小型公司營收年增超過25%的有403家，但同時也有437家年減超過20%。大型公司營收年增超過25%的僅25家，年減超過20%則有46家。顯示企業營收表現分歧，需進一步分析各產業狀況。
*   **產業趨勢：** AI與機器人產業成為市場焦點。相關個股如上銀、大銀微、樺漢、研華、所羅門等受到關注。此趨勢反映了全球自動化趨勢，台灣企業在半導體、精密機械與AI技術的結合下，有望在全球供應鏈中占據關鍵位置。

**4. 新聞資訊分析：**

*   **AI與機器人產業：** 新聞中多次提及AI與機器人產業的發展潛力，並點名相關概念股。投資者可關注此產業的長期發展趨勢。
*   **俄烏戰爭：** 俄烏戰爭的潛在停火，可能影響鋼鐵、散裝航運等重建概念股。
*   **關稅政策：** 川普政府的關稅政策，以及對《晶片法案》

### 🔟 個股分析報告

In [10]:
reply = stock_gpt(stock_id="2330")
print(reply)

[*********************100%%**********************]  1 of 1 completed
<ipython-input-4-3ab454e80453>:10: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  quarterly_revenue_growth = np.round(stock.quarterly_financials.loc["Total Revenue"].pct_change(-1).dropna().tolist(), 2)
<ipython-input-4-3ab454e80453>:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  quarterly_revenue_growth = np.round(stock.quarterly_financials.loc["Total Revenue"].pct_change(-1).dropna().tolist(), 2)
<ipython-input-4-3ab454e80453>:16: FutureWarning: The default fill_method='pad' in Series.pct_change is depr

## 台積電 (2330) 近期趨勢分析報告

**報告日期：** 2025年02月16日

**1. 股價表現分析**

*   **近期股價走勢疲軟：** 從2025年2月5日至2025年2月14日，台積電股價呈現震盪下跌趨勢。由1110元下跌至1060元，跌幅約4.5%。
*   **每日報酬波動大：** 近期每日報酬率顯示股價波動性較高，2月10日出現較大跌幅(-1.78%)，2月14日跌幅亦達-2.75%。
*   **技術指標警訊：** 從新聞資訊中提及的「跌破區間」、「波段回檔股」、「空頭均線下殺」等字眼判斷，技術面可能出現較為不利的訊號，股價可能面臨進一步下探的風險。

**2. 基本面分析**

*   **營收成長穩健：** 2024年各季度營收成長率維持在13%-14%的高水準，顯示公司營運具備成長動能。但2024年Q1營收成長率為-5%，需要關注後續是否持續衰退。
*   **EPS 表現亮眼：** 近期EPS表現優異，從2024年Q2的9.56元成長至2024年Q4的14.45元，顯示獲利能力持續提升。
*   **EPS 季增率趨緩：** EPS季增率從Q2的10%上漲至Q3的31%，到Q4卻下降至15%，需要關注公司後續的獲利能力是否能維持高檔。

**3. 新聞資訊分析**

*   **川普政策風險：** 多則新聞提及川普政府可能對晶片產業採取更強硬的貿易政策，包括恢復對等關稅、修改《晶片法案》等，可能對台積電產生不利影響。
*   **潛在合作機會：** 彭博報導指出，應川普政府要求，台積電可能考慮接管英特爾美國廠，這若能成真，將為公司帶來新的成長機會，但亦存在不確定性。
*   **先進封裝需求強勁：** 日月光投控法說會提及先進封裝需求強勁，這對台積電的先進製程技術及相關業務而言是一大利多。

**4. 綜合評估與趨勢預測**

綜合以上分析，台積電近期面臨以下幾個主要挑戰與機會：

*   **挑戰：**
    *   **總體經濟與地緣政治風險：** 全球經濟情勢不確定性高，加上中美貿易戰、俄烏戰爭等地緣政治風險，可能影響半導體產業需求。
    *   **川普政策風險：** 川普政府的貿易政策可能對台積電的全球布局及獲利能力產生負面影響。
    *   **技術競爭壓力：** 英特爾等競爭對手積極發展先進製程技術，

### 1️⃣1️⃣ 雞蛋水餃股也能做分析

In [11]:
reply = stock_gpt(stock_id="4414")
print(reply)

[*********************100%%**********************]  1 of 1 completed
<ipython-input-4-3ab454e80453>:10: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  quarterly_revenue_growth = np.round(stock.quarterly_financials.loc["Total Revenue"].pct_change(-1).dropna().tolist(), 2)
<ipython-input-4-3ab454e80453>:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  quarterly_revenue_growth = np.round(stock.quarterly_financials.loc["Total Revenue"].pct_change(-1).dropna().tolist(), 2)
<ipython-input-4-3ab454e80453>:16: FutureWarning: The default fill_method='pad' in Series.pct_change is depr

## 如興 (4414) 近期趨勢分析報告

**報告日期：** 2025年02月14日

**1. 前言**

本報告旨在分析如興 (4414) 近期股價走勢、基本面表現及相關新聞資訊，以評估其短期及中期投資趨勢。本報告綜合考量技術面、財務面及事件面等因素，力求提供客觀、嚴謹的分析結果，供投資者參考。

**2. 股價表現分析 (2025/02/05 - 2025/02/14)**

*   **總體趨勢：** 近期如興股價呈現明顯上漲趨勢，從2025年2月5日的 3.25 元，上漲至2025年2月13日的 3.79 元，隨後在2025年2月14日略為回調至 3.74 元。
*   **漲幅：** 區間漲幅達到約15.08% ( (3.74 - 3.25) / 3.25 )，表現強勁。
*   **波動性：** 每日報酬率波動較大，尤其在2025年2月12日及13日，分別達到 3.88% 及 8.91%，顯示股價波動性增強。
*   **技術面觀察：** 2025年2月13日股價一度衝高至 3.79 元，但隨後出現回落，顯示上方可能存在一定壓力。

**表1：近期股價資訊**

| 日期       | 收盤價  | 每日報酬 | 漲跌價差 |
| ---------- | ------- | -------- | -------- |
| 2025-02-05 | 3.25    | NaN      | NaN      |
| 2025-02-06 | 3.26    | 0.31%    | 0.01     |
| 2025-02-07 | 3.28    | 0.61%    | 0.02     |
| 2025-02-10 | 3.31    | 0.91%    | 0.03     |
| 2025-02-11 | 3.35    | 1.21%    | 0.04     |
| 2025-02-12 | 3.48    | 3.88%    | 0.13     |
| 2025-02-13 | 3.79    | 8.91%    | 0.31     |
| 2025-02-14 | 3.74    | -1.32%   | -0.05    |

**3. 基本面分析**

*   **營收成長率：** 近期營收成長率呈現不穩定狀態。